In [30]:
%matplotlib inline
import torch
import torchvision
import detectron2
import fiftyone
import wget
import json
import requests
import predictor
from PIL import Image
import pandas as pd
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

Using Pretrained model for Inference: Code

In [31]:
# get image and reshape it to a conveniant size
# foo = Image.open('images/bus.jpeg')
# foo = foo.resize((640, 480), Image.ANTIALIAS)
# foo.save('images/bus.jpeg', quality=95)

path = 'images/bus.jpeg'
img = cv2.imread(path)

In [32]:
# Create config
cfg = get_cfg()
cfg.MODEL.DEVICE='cpu'
cfg.merge_from_file("/Users/killianvervelle/work/HES-SO/PI_VisionAid/detectron2/detectron2/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"

# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
outputs = predictor(img)

In [33]:
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im = v.get_image()[:, :, ::-1]
cv2.imshow("bus", im)
cv2.imwrite("images/bus_processed.jpg", im)

True

In [115]:
test = outputs["instances"].to("cpu")
test

Instances(num_instances=10, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[ 97.3907, 148.1846, 601.7126, 424.4240],
        [  0.7684, 254.9417,  50.6755, 414.1727],
        [ 60.0165, 245.5184,  96.0513, 404.6560],
        [620.0663, 281.2089, 638.7627, 350.6890],
        [ 81.6516, 262.9363, 117.2575, 411.5894],
        [ 10.8220, 275.9824,  44.2571, 328.6751],
        [ 78.0003, 331.7604,  95.0790, 363.1315],
        [122.5023, 265.3639, 133.1837, 312.1026],
        [ 38.0091, 271.3133,  65.6289, 397.5218],
        [277.2604, 258.5847, 318.8430, 297.2078]])), scores: tensor([0.9987, 0.9908, 0.9836, 0.9835, 0.9664, 0.9595, 0.9472, 0.9434, 0.9104,
        0.8316]), pred_classes: tensor([ 5,  0,  0,  0,  0, 24, 26,  0,  0,  0])])

In [116]:
test_scores = test.scores.tolist()
test_scores

[0.9986549615859985,
 0.9908115863800049,
 0.983623743057251,
 0.9835221767425537,
 0.9663916230201721,
 0.9594699144363403,
 0.9472321271896362,
 0.9433847069740295,
 0.9104428887367249,
 0.8316048979759216]

In [117]:
bind_boxes_x_y = test.pred_boxes.tensor.cpu().numpy()
bind_boxes_x_y = pd.DataFrame(bind_boxes_x_y)
bind_boxes_x_y

,0,1,2,3
0,97.390724,148.184647,601.712585,424.423950
1,0.768425,254.941711,50.675457,414.172729
2,60.016541,245.518402,96.051254,404.655975
3,620.066284,281.208893,638.762695,350.688965
4,81.651619,262.936340,117.257454,411.589355
5,10.822001,275.982422,44.257061,328.675140
6,78.000305,331.760376,95.078972,363.131470
7,122.502281,265.363861,133.183701,312.102600
8,38.009098,271.313324,65.628876,397.521759
9,277.260406,258.584656,318.842987,297.207794


In [118]:
pred_classes = test.pred_classes.tolist()
pred_classes

[5, 0, 0, 0, 0, 24, 26, 0, 0, 0]

In [119]:
classes = pd.read_csv("classes.txt", sep=":").rename(columns={"{0": "class_number", " u'__background__',": "class_name"})
classes['class_name'] = classes['class_name'].str[3:].str[:-2]
classes['class_number'] = classes['class_number'] - 1
classes

,class_number,class_name
0,0,person
1,1,bicycle
2,2,car
3,3,motorcycle
4,4,airplane
...,...,...
75,75,vase
76,76,scissors
77,77,teddy bear
78,78,hair drier


In [120]:
pred_classes_dict = dict()
pred_classes_list = list()
for i in pred_classes:
    pred_classes_dict.update({i:classes.iloc[i, 1]})
    pred_classes_list.append(classes.iloc[i, 1])

In [121]:
pred_classes_dict

{5: 'bus', 0: 'person', 24: 'backpack', 26: 'handbag'}

In [122]:
pred_classes_list

['bus',
 'person',
 'person',
 'person',
 'person',
 'backpack',
 'handbag',
 'person',
 'person',
 'person']

In [124]:
classe_score = pd.concat([pd.DataFrame(
    {'classes': pred_classes_list,
     'scores': test_scores
    }), bind_boxes_x_y], axis=1).rename(columns={0: "x1", 1: "y1", 2: "x2", 3: "y2"})
classe_score

,classes,scores,x1,y1,x2,y2
0,bus,0.998655,97.390724,148.184647,601.712585,424.423950
1,person,0.990812,0.768425,254.941711,50.675457,414.172729
2,person,0.983624,60.016541,245.518402,96.051254,404.655975
3,person,0.983522,620.066284,281.208893,638.762695,350.688965
4,person,0.966392,81.651619,262.936340,117.257454,411.589355
5,backpack,0.959470,10.822001,275.982422,44.257061,328.675140
6,handbag,0.947232,78.000305,331.760376,95.078972,363.131470
7,person,0.943385,122.502281,265.363861,133.183701,312.102600
8,person,0.910443,38.009098,271.313324,65.628876,397.521759
9,person,0.831605,277.260406,258.584656,318.842987,297.207794
